In [ ]:
!unzip /content/drive/MyDrive/dataset.zip

In [1]:
import numpy as np
import pandas as pd
import librosa
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import IPython.display as ipd
from tensorflow.keras.optimizers import SGD
from keras import regularizers
import keras

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.array(active_events)

In [3]:
l = os.listdir('/content/X')
l.sort()
filename_data = pd.DataFrame(l , columns = ['filename'])

l = os.listdir('/content/Y')
l.sort()
event_data= pd.DataFrame(l , columns = ['eventname'])



In [4]:
def r_shape(data):
  data = np.reshape(data,(64,1000))
  data = np.reshape(data,(64,1000,1))
  return data
#def r_shape_2(data):
 # data=np.reshape(data,(1000,11))
  #print(data.shape)
  #return data


In [5]:
data=np.zeros((10000,64,1000,1))
labels=np.zeros((10000,11))
for i,row in filename_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/X'),str(row["filename"]))
  d=np.load(file_name)
  data[i]=r_shape(d)




In [6]:
for i,row in event_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/Y'),str(row["eventname"]))
  e = np.load(file_name)
  e=eventroll_to_multihot_vector(e)
  labels[i]=e

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size=0.20,random_state=42)

In [10]:
x_train=data
y_train=labels
del(data)
del(labels)

In [11]:
input_shape=(64,1000,1)
x_train.shape,y_train.shape

((10000, 64, 1000, 1), (10000, 11))

In [12]:
model =  models.Sequential([
    
                          layers.Conv2D(32 , (3,3),activation = 'relu',padding='valid', input_shape = input_shape),  
                          layers.MaxPooling2D(3, 3),
                          layers.BatchNormalization(),
                          layers.Conv2D(32, (3,3), activation='relu',padding='valid'),
                          layers.Conv2D(32, (3,3), activation='relu',padding='valid'),
                          layers.MaxPooling2D(3, 3),
                          layers.Flatten(),
                          layers.Reshape((8720,2)),
                          layers.LSTM(4),
                          layers.BatchNormalization(),
                          layers.Dropout(0.2),
                          layers.Dense(11 , activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'acc')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 998, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 20, 332, 32)      0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 20, 332, 32)      128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 330, 32)       9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 328, 32)       9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 109, 32)       0

In [ ]:
#history = model.fit(x_train, y_train, epochs=15, validation_data= (x_test, y_test),batch_size=32)
history = model.fit(x_train, y_train, epochs=15, batch_size=32)

In [ ]:
y_pred=model.predict(x_train)

In [ ]:
for i in range(2000):
  for z in range(11):
    if y_pred[i][z]>=0.45:
      y_pred[i][z]=1
    else:
      y_pred[i][z]=0


In [ ]:
from sklearn.metrics import f1_score
k=f1_score(y_test, y_pred, average=None)
print(k)
k=np.mean(k,axis=0)
print(k)